In [ ]:
!pip install transformers -q

     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 182 kB 88.9 MB/s 
     |████████████████████████████████| 7.6 MB 78.4 MB/s 


In [ ]:
!pip install datasets -q

     |████████████████████████████████| 451 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 73.0 MB/s 
     |████████████████████████████████| 115 kB 60.0 MB/s 
     |████████████████████████████████| 127 kB 87.3 MB/s 


In [ ]:
%cd drive/MyDrive/data

/content/drive/MyDrive/data


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, DataCollatorWithPadding

In [ ]:
train_df = pd.read_csv("train.csv")

In [ ]:
train_df['Labels'] = train_df['Target'].map({'neutral':0,
                                 'joy':1,
                                 'surprise':2,
                                 'anger':3,
                                 'sadness':4,
                                 'disgust':5,
                                 'fear':6})

In [ ]:
train_df = train_df.drop(columns = ['ID', 'Speaker', 'Dialogue_ID', "Target"])

In [ ]:
from sklearn.model_selection import train_test_split

t_data, v_data = train_test_split(train_df, test_size = 0.2, random_state = 42,
                                  shuffle = True)

In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(t_data)
val_dataset = Dataset.from_pandas(v_data)

In [ ]:
from datasets.dataset_dict import DatasetDict

raw_data = {
     'train':Dataset.from_dict({'Utterance':train_dataset['Utterance'], 'Labels' : train_dataset['Labels']}),
     'val':Dataset.from_dict({'Utterance':val_dataset['Utterance'], 'Labels' : val_dataset['Labels']})
     }

In [ ]:
raw_data = DatasetDict(raw_data)

### 모델 로드

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = "tae898/emoberta-large"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/408 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
raw_data

DatasetDict({
    train: Dataset({
        features: ['Utterance', 'Labels'],
        num_rows: 7991
    })
    val: Dataset({
        features: ['Utterance', 'Labels'],
        num_rows: 1998
    })
})

In [ ]:
def tokenize_function(example):
	return tokenizer(example['Utterance'],  
                    padding='max_length', 
                    truncation=True, 
                    max_length=128)

In [ ]:
tokenized_datasets = raw_data.map(tokenize_function, batched = True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['Utterance', 'Labels', 'input_ids', 'attention_mask'],
        num_rows: 7991
    })
    val: Dataset({
        features: ['Utterance', 'Labels', 'input_ids', 'attention_mask'],
        num_rows: 1998
    })
})

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["Utterance"])
tokenized_datasets = tokenized_datasets.rename_column("Labels", "labels")
tokenized_datasets.set_format("torch")
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'attention_mask']

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    batch_size=8,
    collate_fn=data_collator,
)

eval_dataloader = DataLoader(
    tokenized_datasets["val"],
    batch_size=8,
    collate_fn=data_collator,
)

In [ ]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 128]),
 'attention_mask': torch.Size([8, 128])}

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

2997


In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

  0%|          | 0/2997 [00:00<?, ?it/s]

### 예측

In [ ]:
test = pd.read_csv('test.csv')

In [ ]:
test = test.drop(columns = ['ID', 'Speaker', 'Dialogue_ID'])

In [ ]:
test_dataset = Dataset.from_pandas(test)

In [ ]:
test_dataset

Dataset({
    features: ['Utterance'],
    num_rows: 2610
})

In [ ]:
test_data = {
     'test':Dataset.from_dict({'Utterance':test_dataset['Utterance']})
     }

test_data = DatasetDict(test_data)

In [ ]:
test_data

DatasetDict({
    test: Dataset({
        features: ['Utterance'],
        num_rows: 2610
    })
})

In [ ]:
tokenized_datasets = test_data.map(tokenize_function, batched = True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/3 [00:00<?, ?ba/s]

In [ ]:
tokenized_datasets = tokenized_datasets.remove_columns(["Utterance"])

In [ ]:
tokenized_datasets.set_format("torch")
tokenized_datasets["test"].column_names

['input_ids', 'attention_mask']

In [ ]:
test_dataloader = DataLoader(
    tokenized_datasets["test"],
    batch_size=8,
    collate_fn=data_collator,
)

In [ ]:
for batch in test_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([8, 128]), 'attention_mask': torch.Size([8, 128])}

In [ ]:
from datasets import load_metric
metric = load_metric('f1')


model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  


KeyError: ignored